In [32]:
import csv
import json
from pprint import pprint
import pandas as pd
from pandas import json_normalize
import pickle
from time import sleep
import requests

In [33]:
key = open("./APIkey.txt", "r").readline()

FileNotFoundError: [Errno 2] No such file or directory: './APIkey.txt'

In [34]:
def get_matchid(puuid, key, n, region="asia"):
    matchid = []
    step = 0
    for i in puuid:
        step += 1
        try:
            request = requests.get(
                f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                timeout=5,
            )
        except requests.Timeout:
            for i in range(5):
                try:
                    request = requests.get(
                        f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                        timeout=5,
                    )
                    break
                except requests.Timeout:
                    pass

        if request.status_code == 429:
            print("Rate limit exceeded!")
            sleep(120)
            try:
                request = requests.get(
                    f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                    timeout=5,
                )
            except requests.Timeout:
                request = requests.get(
                    f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}",
                    timeout=5,
                )
        request = json.loads(request.content)
        print(request)
        matchid.extend(request)
        print(f"step {step} done!")
    return list(set(matchid))

def match_info(match_id, key, region='asia'):
    global game_record
    for i in match_id:
        try:
            request = requests.get(f'https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}', timeout=5,)
        except requests.Timeout:
            for i in range(5):
                try:
                    request = requests.get(f'https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}', timeout=5,)
                    break
                except requests.Timeout:
                    pass

        if request.status_code == 429:
            print("Rate limit exceeded!")
            sleep(120)
            try:
                request = requests.get(f'https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}', timeout=5,)
            except requests.Timeout:
                request = requests.get(f'https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}', timeout=5,)

        request = json.load(request.content)
        request = json_normalize(request)
        game_record = pd.concat([game_record, request])

    return game_record
        

In [27]:
with open(f"./match_id/matchid_{i}.pickle", "rb") as f:
    match_id = pickle.load(f)

print(len(match_id))

3626


In [29]:
matchId_list = []

for i in range(0, 27):
    with open(f"./match_id/matchid_{i}.pickle", "rb") as f:
        match_id = pickle.load(f)
    matchId_list.extend(match_id)

matchId_list = list(set(matchId_list))
matchId_list.sort()
matchId_list = matchId_list[matchId_list.index('KR_6197579703'):]
print(len(matchId_list))

for i in range(1, 30):
    with open(f"./merged_match_id/match_id{i}.pickle", "wb") as f:
        pickle.dump(matchId_list[:i*1000], f)
with open(f"./merged_match_id/match_id{30}.pickle", "wb") as f:
        pickle.dump(matchId_list[29001:], f)


90286


In [35]:
game_record = pd.DataFrame()

with open(f"./match_id/matchid_1.pickle", "rb") as f:
    match_id = pickle.load(f)
res = match_info(match_id, key)

with open(f"./match_data/match_info1.pickle", "wb") as f:
    pickle.dump(res, f)
res.to_csv('match_info1.csv')

NameError: name 'key' is not defined